In [1]:
import re 
from functools import partial

import pandas as pd 
import numpy as np

import torch 
import spacy
import sklearn

from datasets import load_dataset
from datasets import Value, ClassLabel, Features, DatasetDict
from datasets import load_metric
import evaluate

import transformers
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM, AutoModelForSequenceClassification
from transformers import GPT2Tokenizer, GPTNeoForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import logging
from transformers import TrainingArguments, Trainer

from transformers import RobertaForSequenceClassification, BertForSequenceClassification

from preprocessing.cleaning_utils import *
from train_utils.metrics import *
from train_utils.plot_utils import *
from train_utils.custom_trainer import *

import matplotlib.pyplot as plt
import seaborn as sns

from train_utils.metrics import *
from transformers import RobertaForSequenceClassification
from transformers import TrainingArguments, Trainer


In [2]:
import pandas as pd
from datasets import load_dataset
from datasets import Value, ClassLabel, Features, DatasetDict


In [3]:
# pd.read_csv("/home/vs428/project/n2c2/2022/Data/n2c2_track3_test.csv")
# data = pd.read_csv()
DATA_DIR = "/home/vs428/project/n2c2/2022/Data/dev.csv"
OUTPUT_DIR = "/home/vs428/Documents/n2c2_2022/outputs/2022-10-26/03-12-17/outputs/roberta_large/checkpoint-432"
BATCH_SIZE = 16
SPACY_ASSESSMENT =  "/home/vs428/project/n2c2_spacy_models/assessment_model_v2/model-best"
SPACY_PLAN =  "/home/vs428/project/n2c2_spacy_models/plan_subsection_model_v3/model-best"


In [4]:
model = RobertaForSequenceClassification.from_pretrained(OUTPUT_DIR,
                                                        num_labels=4)
tokenizer = AutoTokenizer.from_pretrained("/home/vs428/project/models/RoBERTa-large-PM-M3-Voc/RoBERTa-large-PM-M3-Voc-hf")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
nlp_assessment = spacy.load(SPACY_ASSESSMENT, exclude="parser")
nlp_plan = spacy.load(SPACY_PLAN, exclude="parser")


/gpfs/ysm/project/rtaylor/vs428/conda_envs/n2c2_env3/lib/python3.9/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [6]:

# create hf Dataset
classes = ['Not Relevant', 'Neither', 'Indirect', 'Direct']

# instead we will use the raw text for now
features = Features({
    'ROW ID':Value("int64"),
    'HADM ID':Value("int64"),
    'Assessment':Value("string"),
    'Plan Subsection':Value("string"),
    "Relation":Value("string")
}) 

dataset = load_dataset("csv", data_files={
                            "train":DATA_DIR,
                        },
                       features=features)

# create encoded class labels and rename
dataset = dataset.class_encode_column("Relation")
label2id = {'Not Relevant':3, 'Neither':2, 'Indirect':1, 'Direct':0}
id2label = {v:k for k,v in label2id.items()}
dataset = dataset.align_labels_with_mapping(label2id, "Relation")
dataset = dataset.rename_column("Relation", "label")


Using custom data configuration default-06b7106e5d40a705


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

/gpfs/ysm/project/rtaylor/vs428/conda_envs/n2c2_env3/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:697: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /home/vs428/.cache/huggingface/datasets/csv/default-06b7106e5d40a705/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Aligning the labels:   0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
dataset = dataset.shuffle(seed=42)

In [8]:
dataset = dataset['train'].select(range(100))


In [9]:
# dataset[32]

In [10]:

# add the named entities
dataset = dataset.map(partial(add_ner_assessment, nlp=nlp_assessment))
dataset = dataset.map(partial(add_ner_plan, nlp=nlp_plan))        


  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

In [11]:

# we ASSUME that the ner labels we want are lowercase, UNLIKE the standard ones in the model
spans = [x for x in nlp_plan.get_pipe("ner").labels if x.islower()] + [x for x in nlp_assessment.get_pipe("ner").labels if x.islower()]

tokens = []
for span in spans:
    tokens.append("<" + span + ">")
    tokens.append("</" + span + ">")            

# add the span tags to the vocab
_ = tokenizer.add_tokens(tokens)
model.resize_token_embeddings(len(tokenizer))


Embedding(50017, 1024, padding_idx=1)

In [12]:

# create training args and Trainer
test_args = TrainingArguments(output_dir=OUTPUT_DIR, 
                                    do_train = False,
                                    do_predict = True,
                                    evaluation_strategy="steps",
                                    learning_rate=1e-5,
                                    load_best_model_at_end=True,
                                    warmup_ratio = 0.06,
                                    gradient_accumulation_steps = 8,
                                    per_device_eval_batch_size = BATCH_SIZE,   
                                    dataloader_drop_last = False,  
                                    fp16=True,
                                    log_level="debug",
                                    logging_dir=f"./outputs/test",
                                    logging_strategy="steps",
                                    logging_first_step=True,
                                    logging_steps=500,                                      
                                    # save_steps=1000,
)


In [13]:

# metrics to track
acc = load_metric("accuracy")
macrof1 = load_metric("f1")    
roc_auc_score = evaluate.load("roc_auc", "multiclass")

# create metric_dict for compute_metrics
metric_dict = {}
metric_dict['accuracy'] = {"metric":acc}
metric_dict['f1-macro'] = {"metric":macrof1, "average":"macro"}
metric_dict['auroc'] = {'metric':roc_auc_score, "multi_class":'ovr'}
metric_dict['roc'] = {}
metric_dict["pr"] = {}


/tmp/ipykernel_2445/261045724.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  acc = load_metric("accuracy")


In [14]:
# tokenize
dataset = dataset.map(partial(tokenize_function, tokenizer=tokenizer), batched=True)
print(tokenizer.decode(dataset[0]['input_ids']))



  0%|          | 0/1 [00:00<?, ?ba/s]

Ignored unknown kwarg option direction
<s>88 year old female with hx <secondary_problem>addison</secondary_problem>'s, <secondary_problem>PE</secondary_problem> [**2127**], <secondary_problem>HTN</secondary_problem>, <secondary_problem>mild AS</secondary_problem> s/p recent
   femur fx and <secondary_problem>ORIF</secondary_problem> p/w <primary_symptom>fevers</primary_symptom>, <primary_sign>hypotension</primary_sign>, <primary_problem>PE</primary_problem> and <primary_symptom>elevated troponin</primary_symptom>
   concerning for <primary_problem>sepsis</primary_problem> vs cardiogenic shock.
  .</s></s><problem>Depression</problem>:
   - hold qhs nortriptylline for drowsiness
  .</s>


In [15]:
# cast as pytorch tensors and select a subset of columns we want

dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# create collator
data_collator = DataCollatorWithPadding(tokenizer,
                                        max_length=512, 
                                        padding="longest",
                                        return_tensors="pt")    
# create Trainer
trainer = Trainer(
    model=model,
    args=test_args,
    compute_metrics=partial(compute_metrics, metric_dict=metric_dict),
    data_collator=data_collator,
)    

# predict for metrics
metrics = trainer.predict(dataset)


Using amp half precision backend
The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: ROW ID, Assessment, Plan Subsection, HADM ID.
***** Running Prediction *****
  Num examples = 100
  Batch size = 16


In [16]:

fpr, tpr, roc_auc = metrics[2]['test_roc']
precision, recall, ap = metrics[2]['test_pr']


In [17]:
print(roc_auc), print(ap)

{0: 0.9412336085478388, 1: 0.88878096163186, 2: 0.9364035087719298, 3: 0.9939024390243902}
{0: 0.8704608475399298, 1: 0.7379087539887257, 2: 0.845438509779852, 3: 0.9814814814814816}


(None, None)

In [18]:

print("id2label", id2label)
# plot_multiclass_roc(fpr, tpr, roc_auc, figsize=(8, 6), labels=id2label, fname="Eval_AUROC.png")
# plot_multiclass_pr(precision, recall, ap, figsize=(8, 6), labels=id2label, fname="Eval_AUPRC.png")

preds = np.argmax(metrics.predictions, axis=-1)

# test_dataset['test'].to_csv("test_dataset_output.csv")
# np.savetxt("test_predictions.csv", preds, delimiter=",")
# np.savetxt("test_label_ids.csv", predict_output.label_ids, delimiter=",")

id2label {3: 'Not Relevant', 2: 'Neither', 1: 'Indirect', 0: 'Direct'}


In [19]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [20]:
y_true = dataset['label'].numpy()
y_true, preds

(array([2, 1, 0, 1, 3, 0, 0, 1, 3, 2, 1, 2, 0, 1, 2, 0, 3, 0, 2, 0, 0, 3,
        0, 3, 1, 0, 1, 2, 0, 1, 1, 3, 1, 0, 0, 2, 2, 1, 0, 0, 3, 2, 2, 2,
        2, 2, 0, 3, 1, 3, 2, 1, 3, 0, 2, 1, 1, 1, 1, 2, 1, 0, 3, 2, 1, 2,
        1, 0, 1, 2, 2, 0, 0, 1, 1, 0, 3, 1, 0, 0, 3, 0, 0, 2, 3, 3, 1, 1,
        0, 2, 1, 0, 3, 3, 0, 2, 1, 2, 3, 1]),
 array([2, 1, 0, 1, 3, 0, 0, 1, 3, 2, 1, 2, 0, 1, 2, 0, 3, 0, 2, 2, 0, 3,
        0, 3, 2, 0, 1, 1, 1, 1, 1, 0, 2, 0, 0, 2, 2, 2, 0, 1, 3, 2, 2, 2,
        2, 2, 1, 3, 2, 3, 2, 0, 3, 2, 2, 2, 0, 1, 1, 2, 1, 1, 3, 2, 2, 2,
        1, 2, 1, 2, 2, 0, 0, 1, 0, 0, 3, 1, 0, 0, 3, 0, 0, 2, 3, 3, 1, 1,
        0, 2, 1, 0, 3, 3, 2, 2, 1, 2, 3, 2]))

In [21]:

indirect_wrongs = np.where((y_true == 1) & (y_true != preds))

In [22]:
indirect_wrongs

(array([24, 32, 37, 48, 51, 55, 56, 64, 74, 99]),)

In [23]:
TSET_IDX = 35
y_true[TSET_IDX], preds[TSET_IDX], dataset['Plan Subsection'][TSET_IDX]

(2, 2, '<problem>Panic disorder</problem>: Valium prn CIWA')

In [24]:
indirect_wrongs

(array([24, 32, 37, 48, 51, 55, 56, 64, 74, 99]),)

In [25]:
IDX = indirect_wrongs[0][0]

In [26]:
indirect_wrong_text = []
for x in indirect_wrongs[0]:
    indirect_wrong_text.append(dataset['Assessment'][x] + "<SEP>" + dataset['Plan Subsection'][x])

In [27]:
# indirect_wrong_text[2]

In [28]:
import re 
def drop_spacy_annotations(texts):
    outs = []
    for text in texts:
        outs.append(re.sub(r'<\/?secondary_problem>|<\/?problem>|<\/?primary_problem>|<\/?primary_symptom>|<\/?primary_sign>|<\/?complication_related_to_problem>|<\/?event_related_to_problem>|<\/?organ_failure_related_to_problem>', 
                           '', text))
    return outs

In [29]:
indirect_wrong_text_dropped = drop_spacy_annotations(indirect_wrong_text)

In [30]:
len(dataset['Assessment'])#[32]

100

In [31]:
IDX2 = list(zip(range(len(indirect_wrongs[0])), indirect_wrongs[0]))#[(0,0),(1,7),(2,8),]# (1,10), (2,16), (3, 32), (4, 34)]
IDX1 = 1
for x, y in IDX2:
    print(indirect_wrong_text[x]), print("\n"), print(indirect_wrong_text_dropped[x]), print(f"Predicted: {id2label[preds[y]]}"), print(f"True: {id2label[1]}")
    print("\n\n-----------------------------------------------------\n\n")

Mr. [**Known lastname 2891**] is a 43 y/o M with hx of <secondary_problem>HIV/AIDs</secondary_problem>, off HAART x 6 months
   w/<secondary_problem>CDC 16</secondary_problem> presenting with <primary_symptom>fevers</primary_symptom>, <primary_symptom>dyspnea</primary_symptom> likely [**2-17**] <primary_problem>PCP</primary_problem>.<SEP><problem>Weight loss</problem> - likely multifactorial including <complication_related_to_problem>diarrhea</complication_related_to_problem>, <complication_related_to_problem>infection</complication_related_to_problem>,
   AIDS.
   -encourage po's
   -likely further work up in outpatient setting.


Mr. [**Known lastname 2891**] is a 43 y/o M with hx of HIV/AIDs, off HAART x 6 months
   w/CDC 16 presenting with fevers, dyspnea likely [**2-17**] PCP.<SEP>Weight loss - likely multifactorial including diarrhea, infection,
   AIDS.
   -encourage po's
   -likely further work up in outpatient setting.
Predicted: Neither
True: Indirect


----------------------

In [57]:
fig, ax = plt.subplots(figsize=(12,12))
ax.tick_params(axis='both', which='major', labelsize=13)
ax.tick_params(axis='both', which='minor', labelsize=13)
fig.suptitle('Validation Set Confusion Matrix', fontsize=20, weight="bold")
plt.xlabel('Predicted Label', fontsize=16, weight="bold")
plt.ylabel('True Label', fontsize=16, weight="bold")
# plt.tight_layout()
ConfusionMatrixDisplay.from_predictions(dataset['label'].numpy(), preds, ax=ax, display_labels=['Direct', 'Indirect', 'Neither', 'Not Relevant'])
plt.savefig("conf.png",dpi=300, bbox_inches = "tight")